# Chapter 4 - word based models

The chapter begins with an brief introduction to word based(lexical) translation. It delves into the alignment sub-problem of SMT and how it can be specially challenging for language pairs which do not follow the same order/structure of sentences. It introduces the NULL trick which can be used to link words in the translation which are not present in the root language. 

## IBM Model 1

Sentence-to-sentence translation is useless. There's is not nearly enough support for every sentence pair and it would never generalize to new sentences with a different structure. Thus, Model1 focusses on translating through lexical translation distributions.

### The Problem:
Get the joint probability of the English sentence **e** and the alignment function *a(f)* given the Foreign sentence **f**. The model assumes that each output word *e* in sentence **e** is generated from a single input word *f* in sentence **f**. The alignment function *a(f)* defined for single words of foreign sentence **f** returns the word(position) of the output word *e* in **e**.

The complete generative model is defined as: 
$$ p(\textbf{e}, a | \textbf{f}) = \frac{\epsilon}{(l_f + 1)^{l_e}} \prod_{j=1}^{l_e}t(e_j | f_{a(j)}) $$
where *t()* is the translation probability of an english word given its **aligned** foreign word. 

### The ML Solution:

The model described above is useless without the alignment function. While its relatively easy to find parallel sentence-aligned texts, a word-aligned corpus is impossible to find. The catch however, is that the texts infact are aligned in *some* way - we just need to learn this hidden alignment that from the data. 

***Enter EXPECTATION MAXIMIZATION***
1. Initialize model
2. Apply model params to get the likelihood of data (E)
3. Learn model params from data (M)
4. Repeat 2,3 till convergence

### DIGRESSION: A simple EM example - http://webdocs.cs.ualberta.ca/~nray1/CMPUT466_551/EM_naturebio.pdf

Two labelled coins($A, B$) with known biases are put in a bag. The expriment involves drawing a coin out at random and then recording 10 tosses. In these settings, it is relatively easy to estimate $\theta(\theta_A, \theta_B)$ using MLE - we simply maximize the log-likelihood function:

$$log(L(\theta; x,z)) = log(p(x,z; \theta))$$ where
$$ L: likelihood\ function $$ $$ \theta: parameters$$ $$x: number\ of\ heads$$ $$z: coin\ identity $$

But, how would this problem be solved if ***z*** is hidden? Since the identity of the coin is hidden, we cannot simply count the number of heads as we do not know which series of tosses belong to which coin!

However, this information can be inferred from the data. Since the coin will consistently perform similarly across draws(the performance here refers to the number of heads in 10 tosses), we can try to infer which tosses belong to A and which to B. What this boils down to is a chicken-and-egg problem: if we knew the true identity of the coins, we could accurately estimate the $\theta$ parameters; if we knew the true parameters, we could try to determine the identity of the draws. The EM algorithm is an iterative way of determining the MaxLikelihood estimates of model parameters with hidden data(laten variables).

By beginning with random values of $\theta$, we can determine the "most likely" coin identities of each draw by computing the likelihood of the tosses given the random $\theta$. Since we now know(or at the very least have some idea) of the latent coin identites ***z***, we can compute new estimates of $\theta$ using MLE.

We'll now solve the two coin problem posed in the paper.
<img src="../images/nature_coin_data.png">

The image represents a best-case scenario - one where we know the identity of every draw(***z***). Armed with this info, one can easily estimate $\theta$ using MLE as depicted on the right.

But if the identity is not known, then we are left with estimating the parameters from only the data. Lets begin with an initial estimate of 0.6 and 0.5 for $\theta$. Knowing these, we can find the distribution of the latent variable. $\theta_i$ represents the initial $\theta$ assumption.

$$Q(z) = P(z|x, \theta_i) = 
\frac{P(x, z, \theta_i)}{p(x, \theta_i)} = 
\frac{P(x, z|\theta_i) * P(\theta_i)}{P(x|\theta_i) * P(\theta_i)} = 
\frac{P(x, z\ |\theta_i)}{\sum_zP(x, z|\theta_i)}$$ 

In the expression above, the numerator defines the probability distribution over values of **x, z** given $\theta$, i.e. what is the probability of seeing 5 heads in draw1 when the coin is A, given that the best-estimated parameter of coin A is 0.6. The denominator is the normalizing constant: the sum of probabilities for a given draw.

In [1]:
import numpy as np
import pandas as pd

data = [5, 9, 8, 4, 7] ## random variable x: number of heads for a draw coin
theta_A, theta_B = 0.6, 0.5

In [2]:
def compute_joint_pr(x, theta, n=10):
    return theta**x * (1-theta)**(n-x)

In [3]:
jps = np.zeros((5, 4))

for i, heads in enumerate(data):
    lk_A = compute_joint_pr(heads, theta_A)
    lk_B = compute_joint_pr(heads, theta_B)
    t = lk_A + lk_B
    jps[i, 0] = i+1
    jps[i, 1] = heads
    jps[i, 2] = round(lk_A/t, 3)
    jps[i, 3] = round(lk_B/t, 3)

df = pd.DataFrame(data=jps)
df.columns = ["draw", "x(#heads)", "lk_A", "lk_B"]
print "\nTable 1: E step probabilities"
df


Table 1: E step probabilities


,draw,x(#heads),lk_A,lk_B
0,1.0,5.0,0.449,0.551
1,2.0,9.0,0.805,0.195
2,3.0,8.0,0.733,0.267
3,4.0,4.0,0.352,0.648
4,5.0,7.0,0.647,0.353


Table 1 shows the results of the E step starting with random $\theta$ parameters. Now that we have the probabilities of the drawn coin being A and B, we can find the expected number of heads for coin A and coin B as follows:

$$E[\#heads\ for\ z==A|x, \theta] = h * P(z==A|x, \theta)$$

where $h$ is the number of heads in for draw $x$. The expected values of heads are in Table 2. Since the expected value of sum of random variables is the sum of the expected values of the variables, we can add to get the total expected value of the number of heads contributed by coins A and B across all data.

In [4]:
df['E[#heads z==A]'] = df['lk_A'] * df['x(#heads)']
df['E[#tails z==A]'] = (1 - df['lk_A']) * (10 - df['x(#heads)'])

df['E[#heads z==B]'] = df['lk_B'] * df['x(#heads)']
df['E[#tails z==B]'] = (1 - df['lk_B']) * (10 - df['x(#heads)'])

print df[filter(lambda x:"E" in x, df.columns, )].sum()
print "Table 2: E step expected outcomes"
df

E[#heads z==A]    21.291
E[#tails z==A]     8.431
E[#heads z==B]    11.709
E[#tails z==B]     8.569
dtype: float64
Table 2: E step expected outcomes


,draw,x(#heads),lk_A,lk_B,E[#heads z==A],E[#tails z==A],E[#heads z==B],E[#tails z==B]
0,1.0,5.0,0.449,0.551,2.245,2.755,2.755,2.245
1,2.0,9.0,0.805,0.195,7.245,0.195,1.755,0.805
2,3.0,8.0,0.733,0.267,5.864,0.534,2.136,1.466
3,4.0,4.0,0.352,0.648,1.408,3.888,2.592,2.112
4,5.0,7.0,0.647,0.353,4.529,1.059,2.471,1.941


With the expected values calculated, we have completed the E step of the process. All that's left is the M step, where we re-calculate the new parameter estimates using MLE. 

$$\theta_A = \frac{E[heads, z==A]}{E[heads, z==A] + E[tails, z==A]} = \frac{21.291}{21.291+8.431} = 0.716$$
$$\theta_B = \frac{E[heads, z==B]}{E[heads, z==B] + E[tails, z==B]} = \frac{11.709}{11.709+8.569} = 0.577$$

With the new parameter estimates in hand, we restart the EM process with these new parameters and iterate till convergence. The entire process is displayed very concisely in the image below.
<img src="../images/nature_coin_process.png">

### EM for SMT

das house| the house

das buch | the book

ein buch | a book

The setup is simple. We have a translation corpus of paired french and english sentences. Each pair of samples $k\in[1,n]$ consists of a french sentence $f^k$ and a english sentence $e^k$. Every sentence contains words belonging to its respective language: $f^k_j;j\in[1, m^k]$ for french and $fe^k_i;i\in[1, l^k]$ for english where $m^k,l^k$ are the lengths of sentences for sample pair $k$.

To build a translation model from french to english, we want to generate an english sentence(given a french sentence) which maximizes $p(E=e|F=f)$. 

$$e^* = argmax_{e\in E}\ p(e|f)  \ \ \ \ \ \ \ \ \ \ (1)$$ 

As disussed above, it makes more sense to arrive at sentence $e$ from word-level lexical translations rather than direct sentence based methods - neither do complete sentences repeat that often , nor will this method generalize for new sentences. Now, we could try to directly estimate the conditional probability described in $(1)$ but the actual paper employs a [noisy channel model](https://en.wikipedia.org/wiki/Noisy_channel_model). Instead of maximizing the probability in $(1)$ it expresss the probability as follows using bayes rule:

$$e^* = argmax_{e\in E}\ p(e|f) = \frac{p(f|e) * p(e)}{\sum_{f\in F} p(f|e) * p(e)} \ \ \ \ \ \ \ \ \ \ (2)$$ 

Equation $(2)$ gives us a significant advantage - it allows us to express the optimal translation($e^*$) as a factor of a language model over English. The $p(e)$ term prunes the ill-formed translations since they will have low probabilities and ensures some semblance of correct grammar and sentence construction in the translation process. Directly estimating the first term is difficult (note that it would have been difficult for estimating $p(e|f)$ as well) since we do not know which words map to each other. Hence, the authors introduced a hidden variable for this alignment. Reformulated, the first term can be expressed in terms of the alignment random variable $A$ as:

$$ p(f|e) = \sum_{alignments}p(f, a|e) * p(e) \ \ \ \ \ \ \ \ \ \ (3) $$

As discussed before, the alignment function links a french word to the english word it has generated from. Concretely, if $a(j) = i$ then $f_j$ was generated from $e_i$. Note that $(3)$ precludes the true complexity of the summation - since the alignment function is defined for every french word, the actual expression spans $m$ separate summations, each summing from 0 to $l$ - we include 0 to account for french words which map to no english words.